In [ ]:
"""
import numpy as np
from qiskit import *
%matplotlib inline
from math import pi, sqrt
import random
from qiskit.visualization import plot_histogram
"""

In [1]:
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#pip install scikit-learn

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

## Load Data

In [2]:
X, y = load_iris(return_X_y=True)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100] #0: 0-49, 1:50-99, 2:100-149
y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [6]:
n_qubits = len(X_train[0])
print(X_train[0])
n_qubits

[ 0.35866332  2.73124718 -0.94364311 -0.68644165]


4

## Quantum Kernel Function

In [8]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1 #M observable from our lecture

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [14]:
kernel(X_train[0], X_train[0])

tensor(1., requires_grad=True)

## Quantum Support Vector Machine

In [16]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [17]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [18]:
predictionsTrain = svm.predict(X_train) #SVM that we trained, will predict result from X_train
accuracy_score(predictionsTrain, y_train) #Compare SVM prediction with ACTUAL result, y_train

1.0

In [19]:
predictions = svm.predict(X_test) #Data SVM did not see, SVM will make prediction
accuracy_score(predictions, y_test) #Result from New Prediction from ACTUAL result, y_train

1.0